# https://www.youtube.com/watch?v=EujccFRio7o

"""
Unable to upload lots of images to Github.
Create your own set of images by downloading from Google. 
"""

In [ ]:
#!pip install google_images_download

In [ ]:
'''from google_images_download import google_images_download
response=google_images_download.googleimagesdownload()
arguments={"keywords":"national park, dog park", "limit":100,"print_urls":False}
paths=response.download(arguments)

print(paths)'''

'from google_images_download import google_images_download\nresponse=google_images_download.googleimagesdownload()\narguments={"keywords":"national park, dog park", "limit":100,"print_urls":False}\npaths=response.download(arguments)\n\nprint(paths)'

In [ ]:
#!wget https://valuestockphoto.com/freehighresimages/sample_cart.zip -P downloads

In [ ]:
#!unzip downloads/sample_cart.zip

In [ ]:
from keras.layers import Conv2D, UpSampling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
from skimage.io import imsave, imshow

In [ ]:
# resimlerin path klasöründe değil onun alt klasöründe olması gerek
path = 'drive/MyDrive/data/resimler'

#Normalize images - divide by 255
train_datagen = ImageDataGenerator(rescale=1. / 255)

#Resize images, if needed
train = train_datagen.flow_from_directory(path, 
                                          target_size=(256, 256), 
                                          batch_size=340, 
                                          class_mode=None)

Found 355 images belonging to 1 classes.


In [ ]:
#Convert from RGB to Lab
"""
by iterating on each image, we convert the RGB to Lab. 
Think of LAB image as a grey image in L channel and all color info stored in A and B channels. 
The input to the network will be the L channel, so we assign L channel to X vector. 
And assign A and B to Y.
"""

X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0]) 
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

(340, 256, 256, 1)
(340, 256, 256, 2)


In [ ]:
#Encoder

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

#Decoder
#Decoder
#Note: For the last layer we use tanh instead of Relu. 
#This is because we are colorizing the image in this layer using 2 filters, A and B.
#A and B values range between -1 and 1 so tanh (or hyperbolic tangent) is used
#as it also has the range between -1 and 1. 
#Other functions go from 0 to 1.
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 256)       295168    
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 256)       590080    
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 512)       1180160   
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 512)       2

In [ ]:

model.fit(X,Y,validation_split=0.1, epochs=5, batch_size=16)

model.save('other_files/colorize_autoencoder.model')

#epoch 0 girildiği için aslında eğitim yapılmıyor. En az 300 epoch eğitim gerekli


"\nmodel.fit(X,Y,validation_split=0.1, epochs=0, batch_size=16)\n\nmodel.save('other_files/colorize_autoencoder.model')\n\n#epoch 0 girildiği için aslında eğitim yapılmıyor. En az 300 epoch eğitim gerekli\n"

In [ ]:

###########################################################
#Load saved model and test on images.
#colorize_autoencoder300.model is trained on 300 epochs
#
#
#aşağıda 300 epoch eğitilmiş ve kaydedilmiş model yüklenip kullanılmıştır.
#eğitilmiş modeli çalıştırmak için model nesnesini hiç oluşturmamış olmak gerekiyor
'''
tf.keras.models.load_model(
    'drive/MyDrive/Colab Notebooks/ImageProcessing/AutoEncoders/colorize_autoencoder.model',
    custom_objects=None,
    compile=True)

'''


img1_color=[]
img1=img_to_array(load_img('drive/MyDrive/data/resimler/manzara/016.jpg'))
img1 = resize(img1 ,(256,256))
img1_color.append(img1)
img1_color = np.array(img1_color, dtype=float)
img1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]
img1_color = img1_color.reshape(img1_color.shape+(1,))
output1 = model.predict(img1_color)
output1 = output1*128
result = np.zeros((256, 256, 3))
result[:,:,0] = img1_color[0][:,:,0]
result[:,:,1:] = output1[0]
imsave("result.png", lab2rgb(result))
imshow(lab2rgb(result))



AttributeError: ignored